In [1]:
if 'google.colab' in str(get_ipython()):
  !git clone https://github.com/system205/PMLDL_Assignment1.git
  %cd /content/PMLDL_Assignment1/notebooks

Cloning into 'PMLDL_Assignment1'...
remote: Enumerating objects: 189, done.
remote: Counting objects: 100% (171/171), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 189 (delta 64), reused 149 (delta 44), pack-reused 18
Receiving objects: 100% (189/189), 42.95 MiB | 33.73 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/PMLDL_Assignment1/notebooks


# Install requirements

In [1]:
root='../'
project_path='/workspaces/PMLDL_Assignment1'

if 'google.colab' in str(get_ipython()):
    project_path = '/content/PMLDL_Assignment1'


!pip install -r ../requirements.txt

# Prepare data

### Preprocess

In [5]:
!python ../src/data/preprocess_dataset.py --root={root}

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
577760


### Filter and split on train and validate

In [5]:
!python ../src/models/filter_split_train_val.py --root={root}

# Test simple models - hypothesis

## Test baseline model to compare all other metrics with

### Initial dataset

In [24]:
!python ../src/models/metric/compute_metric.py --dataframe_file={root}data/raw/filtered.tsv --predictions_column=reference --separator='\t'

2023-10-29 13:29:51.028248: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-29 13:29:51.028308: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-29 13:29:51.032495: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-29 13:29:52.125568: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/workspaces/PMLDL_Assignment1/notebooks/../src/mod

### Preprocessed dataset

In [26]:
!python ../src/models/metric/compute_metric.py --dataframe_file={root}data/interim/preprocessed.csv --predictions_column=reference

2023-10-23 18:32:29.174844: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-23 18:32:29.174898: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-23 18:32:29.174960: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-23 18:32:29.181973: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/workspaces/PMLDL_Assignment1/notebooks/../src/mod

### Train dataset

In [1]:
!python ../src/models/metric/compute_metric.py --dataframe_file={root}data/interim/train.csv --predictions_column=reference

2023-10-29 13:41:38.219448: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-29 13:41:38.219505: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-29 13:41:38.219548: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-29 13:41:38.226283: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/workspaces/PMLDL_Assignment1/notebooks/../src/mod

### Validation dataset

In [28]:
!python ../src/models/metric/compute_metric.py --dataframe_file={root}data/interim/val.csv --predictions_column=reference

2023-10-29 13:42:05.405516: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-29 13:42:05.405581: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-29 13:42:05.405639: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-29 13:42:05.412453: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/workspaces/PMLDL_Assignment1/notebooks/../src/mod

## Result

### As you can see the highest score (37%) is on the filtered data: train or val datasets. So, the final solution will be compared with it.

## Test hypothesis 1 model (simple removal of swearing words)

### Fit the train data

In [31]:
!python ../src/models/train_simple_model.py --project_path={project_path} --root={root}

Total number of toxic words to be filtered: 2858


### Evaluate on validation dataset

In [30]:
!python ../src/models/predict_simple_model.py --dataframe_file=../data/interim/val.csv --root={root}
!python ../src/models/metric/compute_metric.py --dataframe_file=../data/interim/val.csv

Number of bad words to be filtered: 2858
2023-10-29 13:42:18.300705: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-29 13:42:18.300770: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-29 13:42:18.300829: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-29 13:42:18.307669: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/workspac

## The result of simple solution is quite good. It is 8% more than of baseline

# Final model - T5 fine-tuning

In [6]:
!python ../src/models/train_model.py --train_path={root}data/interim/train.csv --val_path={root}data/interim/val.csv --save_dir={root}/models/output_dir/

2023-10-30 14:41:24.754229: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-30 14:41:24.754283: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-30 14:41:24.754324: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-30 14:41:28.323562: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Map: 100% 31513/31513 [00:04<00:00, 6457.59 examples/s]
Map: 100% 7879/7879 [00:00<00:00, 12947.57 examples/s]
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than us

### Result:
BLEU score on validation dataset is **53%**.

That is even higher than simple swearing removal on 8%. Meaning the model tries to paraphrase almost completing the task as it was required.

# Test fine tuned model

### Download weights

### Comment one of 3 given texts or write yours

In [2]:
import sys
module_path = f'{project_path}/src/models/'
sys.path.append(module_path)  

from predict_model import predict

texts =  ["Fucking niggers invade France and murder white gays", 
          "Suck my big, and beautiful dick.", 
          "It didn't fuck him."]

predict(root, texts)

2023-11-01 14:18:57.748045: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-01 14:18:57.748111: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-01 14:18:57.748154: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-01 14:18:57.755917: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


/workspaces/PMLDL_Assignment1/src


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Translation: they invade france and they murder white people | Input: fucking niggers invade france and murder white gays
Translation: oh my big and beautiful pet | Input: suck my big and beautiful dick
Translation: it didn t mess with him | Input: it didn t fuck him


# Visualization

In [3]:
from IPython.display import SVG, display, HTML
def svg(source, title):
    display(HTML(f'<h3>{title}</h3><img src="' + source + '" width=500 height=350/>'))

In [4]:
svg(f'{root}reports/figures/training_plots/eval_bleu.svg', 'BLEU score on validation dataset')
svg(f'{root}reports/figures/training_plots/eval_loss.svg', 'Validation loss')
svg(f'{root}reports/figures/training_plots/train_loss.svg', 'Training loss')
svg(f'{root}reports/figures/training_plots/train_learning_rate.svg', 'Learning rate')